##### System Libraries

In [1]:
import os
import subprocess
import logging
import re
import copy
import time
import math

##### Required Libraries

In [2]:
try:
    import scapy
except ModuleNotFoundError:
    os.system('pip install scapy')
    import scapy
try:
    import tqdm
except ModuleNotFoundError:
    os.system('pip install tqdm')
    import tqdm
try:
    import requests
except ModuleNotFoundError:
    os.system('pip install requests')
    import requests
logging.getLogger("requests").setLevel(logging.INFO)
logging.getLogger("urllib3").setLevel(logging.INFO)

import scapy.plist
from scapy.all import rdpcap
from tqdm import trange,tqdm

### Analyzer

#### Minor Function Breakdowns
---
##### Generate_Logger
Generates a logger with default parameters for logging critical components and debugging
##### Generate Self IPs
Generates a list of IPs assigned to current device according to a protocol list.
##### Remove Reserved IPs
Removes IPS that belong in the reserved ranges from the list
##### List Initializer [V2]
Helper function to shorten and simplify initializing and re-initializing lists with values
##### PCAP_Input
Takes a pcap file as input and returns and stores it in memory
##### Generate Unique IPs
Generates a list of unique IPs found in the PCAP data (excluding self IPs)
##### Query Unique IPs
Requests IP information from ip-api through requests, and stores the received data in a list
##### Generate Summary
Summarizes the data in a neater form from ip-api
##### Trace IPs
Traces the IPs and returns the output

In [3]:

class Analyzer:
    if True:
        #Initialization of the logger
        logger=logging.getLogger
        stored_logger_level=[logging.INFO]
        stored_protocol_list=["IPv4","IPv6","Gateway"]

        stored_pcap_data=[]

        stored_self_ips=[]
        stored_unique_ips=[]
        stored_blocked_ips=[]

        stored_queried_ips=[]
    def __init__(self,path,protocol_list=stored_protocol_list,query=True,query_amount=20):
        if True:
            self.logger=self.Generate_Logger(filename='AnalysisCore.log')
            logging.getLogger("requests").setLevel(logging.INFO)
            self.Generate_Self_IPs(protocol_list=protocol_list)
            self.PCAP_Input(path)
            self.Generate_Unique_IPs()
            if query==True:
                self.Query_Unique_IPs(amount=query_amount)
    def Generate_Logger(self,filename='default_logger_name.log'): 
        logging.basicConfig(filename=filename,format='%(asctime)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level[0])
        logger=logging.getLogger()
        logger.debug('Logger initialized.')
        return logger
    def List_Initializer(self,input_list,input_param):
        if isinstance(input_param,list) or isinstance(input_param,scapy.plist.PacketList):
            input_list.clear()
            for item in input_param:
                input_list.append(item)
        else:
            input_list.clear()
            input_list.append(input_param)
    def Generate_Self_IPs(self,protocol_list=stored_protocol_list):
        self.logger.info("Generating self IPs with protocols, "+str(protocol_list))
        self_ips=[]
        process=subprocess.Popen(['ipconfig'],stdout=subprocess.PIPE,text=True,shell=True)
        for line in process.stdout:
            for protocol in protocol_list:
                if re.search(protocol,line.strip()):
                    self_ips.append(line.strip()[line.strip().find(':')+2:])
        self.List_Initializer(self.stored_self_ips,self_ips)
        self.logger.debug('Stored self IPs in stored_self_ips '+str(self_ips))
        return self_ips
    def Remove_Reserved_IPs(self,ips):
        self.logger.debug(f'Removing reserved IP ranges from list')
        patterns=['10 .']+[f'100.{x}.' for x in range(64,128)]+['127.','169.254.']+[f'172.{x}.' for x in range(16,32)]+['192.0.0.','192.168.']+[f'192.{x}.' for x in range(18,20)]+['192.51.100.','203.0.113.']+[f'{x}.' for x in range(224,240)]+['255.','256.']
        #time.sleep(2)
        self.logger.debug(patterns)
        temp=[]
        for ip in ips:
            for pattern in patterns:
                if re.match(pattern,ip):
                    self.logger.debug(f'Matched {ip} with {pattern}')
                    try:
                        ips.remove(ip)
                    except ValueError:
                        self.logger.error(f'Error when trying to remove IP {ip}, from list with pattern {pattern}')
                    temp.append(ip)
                    break
        self.logger.info(f'Removed {temp}')
        return ips
    def Query_IP(self,ip):
        self.logger.debug('Querying for IP '+ip)
        response=requests.get(f"http://ip-api.com/json/{ip}")
        data=response.json()
        return data
    def PCAP_Input(self,path):
        self.logger.info(f'Reading file from path, "{path}"')
        file_size=os.path.getsize(path)
        print(f'Reading PCAP Data. [{math.ceil(file_size/math.pow(1024,2))} MB]')
        pcap_data=rdpcap(path)
        self.logger.debug('Stored '+path+' data in stored_pcap_data.')
        self.List_Initializer(self.stored_pcap_data,pcap_data)
        return pcap_data
    def Generate_Unique_IPs(self,pcap_data=stored_pcap_data,self_ips=stored_self_ips):
        self.logger.info('Extracting Unique IPs from PCAP Data.')
        unique_ips=[]
        blocked_ips=self_ips.copy()
        packets=pcap_data
        for packet in tqdm(packets,desc='Analyzing packets for unique IPs'):
            flag=0
            try:
                for ip in blocked_ips:
                    if packet.payload.dst==ip:
                        flag=1
                if flag==0:
                    blocked_ips.append(packet.payload.dst)
                    unique_ips.append(packet.payload.dst)
            except AttributeError:
                broadcast_temp='256.256.256.256'
                for ip in blocked_ips:
                    if ip==broadcast_temp:
                        flag=1
                if flag==0:
                    blocked_ips.append(broadcast_temp)
                    unique_ips.append(broadcast_temp)
        self.Remove_Reserved_IPs(unique_ips)
        self.List_Initializer(self.stored_unique_ips,unique_ips)
        self.logger.debug('Stored unique IPs in stored_unique_ips.')
        return unique_ips
    def Query_Unique_IPs(self,unique_ips=stored_unique_ips,amount=20):
        self.logger.info(f'Querying unique IPs.')
        queried_unique_ips=[]
        for ip in tqdm(unique_ips[:min(amount,45)],desc='Querying IPs'):
            response=self.Query_IP(ip)
            queried_unique_ips.append(response)
            # self.logger.debug(f'Stored query in stored_queried_unique_ips.')
            self.List_Initializer(self.stored_queried_ips,queried_unique_ips)
        return queried_unique_ips
    def Generate_Summary(self,data=stored_queried_ips):
        self.logger.info(f'Summarizing data.')
        summarized=[]
        for query in tqdm(data,desc='Summarizing data'):
            temp={}
            if query['status']=='success':
                temp['Location']=f"{query['city']}, {query['regionName']}, {query['country']} | Lat: {query['lat']}, Long: {query['lon']}"
                temp['Organization']=f"{query['as']}, {query['org']}, {query['isp']}"
                temp['IP']=f"{query['query']}"
            elif query['status']=='fail':
                temp['Location']=f'Unknown [Failed]'
                temp['Organization']=f'Unknown [Failed]'
                temp['IP']=f"{query['query']}"
            summarized.append(temp)
        self.logger.debug('Summarizing completed.')
        return summarized
    def Trace_IPs(self,ips=stored_unique_ips,amount=99,hops=20,timeout=1000):
        self.logger.info(f'Tracing IPs {ips[:amount]}')
        processes=[]
        for ip in ips[:amount]:
            processes.append(subprocess.Popen(["tracert","-h",str(hops),"-w",str(timeout),ip],
                                              stdout=subprocess.PIPE,text=True,shell=True))
        time.sleep(math.ceil(hops*(timeout/1000)*3.75*(1+amount*0.001)))
        outputs_temp=[]
        for process in processes:
            outputs_temp.append(process.stdout)
        output_cleaned=[]
        for output in outputs_temp:
            temp=[]
            for line in output:
                if line.strip()!='':
                    temp.append(line.strip())
            output_cleaned.append(temp)
        return output_cleaned

In [4]:
A=Analyzer('PCAP Data/720S-Down-Spo-Misc-Idling.pcap',query=True,query_amount=40)

Reading PCAP Data. [4 MB]


Querying IPs: 100%|██████████| 40/40 [00:26<00:00,  1.50it/s]


In [5]:
A.Generate_Summary()

Summarizing data: 100%|██████████| 40/40 [00:00<00:00, 253050.02it/s]


[{'Location': 'Toronto, Ontario, Canada | Lat: 43.6532, Long: -79.3832',
  'Organization': 'AS13335 Cloudflare, Inc., Cloudflare, Inc., Cloudflare, Inc.',
  'IP': '162.159.134.234'},
 {'Location': 'Kansas City, Missouri, United States | Lat: 39.0997, Long: -94.5785',
  'Organization': 'AS15169 Google LLC, Google Cloud, Google LLC',
  'IP': '35.186.224.45'},
 {'Location': 'Stockholm, Stockholm, Sweden | Lat: 59.3293, Long: 18.0686',
  'Organization': 'AS16509 Amazon.com, Inc., AWS EC2 (eu-north-1), Amazon.com, Inc.',
  'IP': '51.21.215.65'},
 {'Location': 'Maharashtra, Maharashtra, India | Lat: 19.5, Long: 76',
  'Organization': 'AS32590 Valve Corporation, Valve Corporation, Valve Corporation',
  'IP': '155.133.224.23'},
 {'Location': 'Kansas City, Missouri, United States | Lat: 39.0997, Long: -94.5785',
  'Organization': 'AS396982 Google LLC, Google Cloud, Google LLC',
  'IP': '34.110.207.168'},
 {'Location': 'Kansas City, Missouri, United States | Lat: 39.0997, Long: -94.5785',
  'Org

In [ ]:
A.Trace_IPs()

In [ ]:
summarized_trace=[]
pattern=re.escape('[')
for trace in outputs_trace:
    temp=[]
    for line in trace:
        match=re.match(r"\d",line.strip())
        if match:
            temp2=line.strip()[30:]
            if re.search(pattern,temp2):
                temp2=temp2[temp2.find('[')+1:temp2.find(']')]
            temp.append(temp2)
    summarized_trace.append(temp)
summarized_trace

In [ ]:

# unique_ips=[]
# temp=[]
# ip=A.stored_unique_ips[1]
# response=requests.get(f"http://ip-api.com/json/{ip}")
# data=response.json()
# temp.clear()
# temp.append(ip)
# temp.append(data)
# unique_ips.append(temp)
# unique_ips